**Preparing the data**

In [1]:
import random
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.utils import text_dataset_from_directory
from tensorflow.keras.layers import TextVectorization, Bidirectional, LSTM, Dropout, Dense, Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [2]:
batch_size = 8

In [3]:
train = text_dataset_from_directory("aclImdb/train", batch_size=batch_size)
validation = text_dataset_from_directory("aclImdb/val", batch_size=batch_size)
test = text_dataset_from_directory("aclImdb/test", batch_size=batch_size)
text_only_train = train.map(lambda x, y: x)

Found 70000 files belonging to 3 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [5]:
max_length = 600
max_tokens = 20000

text_vectorization = TextVectorization(max_tokens=max_tokens, output_mode="int", output_sequence_length=max_length)
text_vectorization.adapt(text_only_train)

In [6]:
int_train_ds = train.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = validation.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

**Model that uses an `Embedding` layer trained from scratch**

In [7]:
inputs = Input(shape=(None,), dtype="int64")
embedded = Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = Bidirectional(LSTM(32))(embedded)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)
model = Model(inputs, outputs)

In [8]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               73984     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
___________________________________________________

In [9]:
callbacks = [ModelCheckpoint("word_embedding_bidir.keras", save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
8750/8750 [==============================] - 506s 57ms/step - loss: -135.3166 - accuracy: 0.1429 - val_loss: 231.8292 - val_accuracy: 0.5000
Epoch 2/10
8750/8750 [==============================] - 474s 54ms/step - loss: -395.6154 - accuracy: 0.1429 - val_loss: 459.5353 - val_accuracy: 0.5000
Epoch 3/10
8750/8750 [==============================] - 463s 53ms/step - loss: -654.5818 - accuracy: 0.1429 - val_loss: 686.8358 - val_accuracy: 0.5000
Epoch 4/10
8750/8750 [==============================] - 463s 53ms/step - loss: -914.1788 - accuracy: 0.1429 - val_loss: 914.1685 - val_accuracy: 0.5000
Epoch 5/10
8750/8750 [==============================] - 471s 54ms/step - loss: -1174.4249 - accuracy: 0.1429 - val_loss: 1141.9229 - val_accuracy: 0.5000
Epoch 6/10
8750/8750 [==============================] - 492s 56ms/step - loss: -1435.1752 - accuracy: 0.1429 - val_loss: 1369.0002 - val_accuracy: 0.5000
Epoch 7/10
8750/8750 [==============================] - 466s 53ms/step - loss: -1695

In [11]:
model = load_model("word_embedding_bidir.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

3125/3125 [==============================] - 80s 25ms/step - loss: 231.8313 - accuracy: 0.5000
Test acc: 0.500
